In [ ]:
import requests
import json
import pandas as pd
# Baba's id
testid=330626



In [ ]:
# each year has a specific id that is passed as a token when requesting data on payments
year_ids={
    2015:'3526a13d-5d1c-5de1-bdc7-d298703270cf',
    2016:'47fcc5d3-e7f7-5b49-870e-44f002f8cdcf',
    2017:'eeca1e54-a309-5a62-8203-9b88de15f31f',
    2018:'9dfd9047-64b2-5b52-8fe2-48f280859693',
    2019:'3ac03130-dafb-5a7b-89fb-2b2790c05f26',
    2020:'93d9435c-cf1f-56fb-83f4-71824fbdf8ec',
    2021:'7dfb0fb2-e573-5448-a968-67899434ce8b'
    }
year_ids[2015]

In [ ]:
# testid=13025
# Above works for 2017
# each year has a specific ID
def doctorSearch(id):
    paymentDump=[]
    for year in range(2015,2022):
        token=year_ids[year]
        doctor_page_url=f'https://openpaymentsdata.cms.gov/api/1/datastore/query/{token}?keys=true&limit=100&offset=0&conditions%5B0%5D%5Bproperty%5D=covered_recipient_profile_id&conditions%5B0%5D%5Bvalue%5D={id}&conditions%5B0%5D%5Boperator%5D=%3D&sorts%5B0%5D%5Bproperty%5D=total&sorts%5B0%5D%5Border%5D=desc&properties%5B0%5D%5Bresource%5D=t&properties%5B0%5D%5Bproperty%5D=applicable_manufacturer_or_applicable_gpo_making_payment_name&properties%5B1%5D%5Bresource%5D=t&properties%5B1%5D%5Bproperty%5D=nature_of_payment_or_transfer_of_value&properties%5B2%5D%5Bresource%5D=t&properties%5B2%5D%5Bproperty%5D=date_of_payment&properties%5B3%5D%5Bresource%5D=t&properties%5B3%5D%5Bproperty%5D=applicable_manufacturer_or_applicable_gpo_making_payment_id&properties%5B4%5D%5Bresource%5D=t&properties%5B4%5D%5Bproperty%5D=record_id&properties%5B5%5D%5Balias%5D=total&properties%5B5%5D%5Bexpression%5D%5Boperator%5D=%2B&properties%5B5%5D%5Bexpression%5D%5Boperands%5D%5B0%5D%5Bresource%5D=t&properties%5B5%5D%5Bexpression%5D%5Boperands%5D%5B0%5D%5Bproperty%5D=total_amount_of_payment_usdollars&properties%5B5%5D%5Bexpression%5D%5Boperands%5D%5B1%5D=0'
        loaded=json.loads(requests.get(doctor_page_url).text)
        if 'status' not in loaded:
            paymentDump =paymentDump + loaded['results']
    # create a dataframe out of the payments
    return pd.DataFrame.from_dict(paymentDump)


- the cache id isn't device dependent

In [ ]:
doctorSearch(460800)

In [ ]:
zipcode = 75227
zipcode_URL=f'https://openpaymentsdata.cms.gov/api/1/datastore/query/d0ca4ed9-0a67-5e9e-9a08-fab7b975d61c?keys=true&limit=100&offset=0&conditions%5B0%5D%5Bproperty%5D=covered_recipient_profile_zipcode&conditions%5B0%5D%5Bvalue%5D={zipcode}%25&conditions%5B0%5D%5Boperator%5D=LIKE&sorts%5B0%5D%5Bproperty%5D=covered_recipient_profile_last_name&sorts%5B0%5D%5Border%5D=asc&sorts%5B1%5D%5Bproperty%5D=covered_recipient_profile_first_name&sorts%5B1%5D%5Border%5D=asc&sorts%5B2%5D%5Bproperty%5D=covered_recipient_profile_middle_name&sorts%5B2%5D%5Border%5D=asc'
dump = json.loads(requests.get(zipcode_URL).text)
id = dump['results'][2]['covered_recipient_profile_id']
print(dump)

In [ ]:
limit = 500
state ='TX'
doctorDF=0
for i in range(202):
    offset =i*limit
    doctorListURL=f'https://openpaymentsdata.cms.gov/api/1/datastore/query/d0ca4ed9-0a67-5e9e-9a08-fab7b975d61c?keys=true&limit={limit}&offset={offset}&conditions%5B0%5D%5Bproperty%5D=covered_recipient_profile_state&conditions%5B0%5D%5Bvalue%5D={state}&conditions%5B0%5D%5Boperator%5D=%3D&sorts%5B0%5D%5Bproperty%5D=covered_recipient_profile_last_name&sorts%5B0%5D%5Border%5D=asc&sorts%5B1%5D%5Bproperty%5D=covered_recipient_profile_first_name&sorts%5B1%5D%5Border%5D=asc&sorts%5B2%5D%5Bproperty%5D=covered_recipient_profile_middle_name&sorts%5B2%5D%5Border%5D=asc'
    newDoctorList=(json.loads(requests.get(doctorListURL).text))['results']
    newDF=pd.DataFrame.from_dict(newDoctorList)[['covered_recipient_profile_primary_specialty','covered_recipient_profile_id','covered_recipient_npi','covered_recipient_profile_zipcode','covered_recipient_profile_first_name','covered_recipient_profile_last_name','covered_recipient_profile_type']]
    if i == 0:
        doctorDF = newDF
    else:
        doctorDF=doctorDF.append(newDF)

In [ ]:
doctorDF.reset_index(inplace = True)
del doctorDF['index']
doctorDF

In [ ]:
doctorDF.reset_index(inplace = True)
del doctorDF['index']
doctorDF.to_csv('Texas Doctor Data.csv')